# Métodos Montecarlo Fing 2022 - Entrega 1

**Autor:** Carlos M. Martinez, marzo-abril 2022.

**Email:** carlosm@fing.edu.uy, carlos@cagnazzo.uy



Supongamos que para construir una casa debemos efectuar la siguiente lista de tareas:

- T1 - cimientos - tiempo aleatorio uniforme entre 40 y 56 hs. 
- T2 - contrapiso - tiempo aleatorio uniforme entre 24 y 32 hs. 
- T3 - paredes - tiempo aleatorio uniforme entre 20 y 40 hs.
- T4 - techo - tiempo aleatorio uniforme entre 16 y 48 hs.
- T5 - instalación sanitaria - tiempo aleatorio uniforme entre 10 y 30 hs.
- T6 - instalación el eléctrica - tiempo aleatorio uniforme entre 15 y 30 hs. 
- T7 - cerramientos - tiempo aleatorio uniforme entre 20 y 25 hs.
- T8 - pintura - tiempo aleatorio uniforme entre 30 y 50 hs.
- T9 - revestimientos sanitarios - tiempo aleatorio uniforme entre 40 y 60 hs.
- T10 - limpieza final - tiempo aleatorio uniforme entre 8 y 16 hs.

Hay ciertas dependencias que implican que una tarea no puede comenzar hasta haberse terminado otra previa:

- T2, T3 dependen de 1.
- T4 depende de 2 y 3
- T5 depende de 2 y 3
- T6 depende de 3
- T7 depende de 3
- T8 dependede 4,5,6 y7
- T9 depende de 5
- T10 depende de 7, 8 y 9

## Entrega 1 

**Ejercicio 2.1**

Implementar un programa que reciba como par ́ametros de l ́ınea de comando (o pregunte en pantalla) la cantidad de replicaciones n a realizar, y emplee Monte Carlo para calcular (e imprimir) la estimación del tiempo total promedio desde que se comienza la obra hasta que se finaliza la misma, y la desviación estándar de este estimador.

In [29]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
from time import perf_counter

# Estructura de datos para las tareas:

TareasDB = ['']*11

TareasDB[1] = {
    'dur': (40,56), # intervalo de duración en horas
    'dep': []       # tareas de la que depende para poder comenzar
}

TareasDB[2] = {
    'dur': (24,32), # intervalo de duración en horas
    'dep': [1]       # tareas de la que depende para poder comenzar
}

TareasDB[3] = {
    'dur': (20,40), # intervalo de duración en horas
    'dep': [1]       # tareas de la que depende para poder comenzar
}

TareasDB[4] = {
    'dur': (16,48), # intervalo de duración en horas
    'dep': [2,3]       # tareas de la que depende para poder comenzar
}

TareasDB[5] = {
    'dur': (10,30), # intervalo de duración en horas
    'dep': [2,3]       # tareas de la que depende para poder comenzar
}

TareasDB[6] = {
    'dur': (15,30), # intervalo de duración en horas
    'dep': [3]       # tareas de la que depende para poder comenzar
}

TareasDB[7] = {
    'dur': (20,25), # intervalo de duración en horas
    'dep': [3]       # tareas de la que depende para poder comenzar
}

TareasDB[8] = {
    'dur': (30,50),        # intervalo de duración en horas
    'dep': [4,5,6,7]       # tareas de la que depende para poder comenzar
}

TareasDB[9] = {
    'dur': (40,60),        # intervalo de duración en horas
    'dep': [5]       # tareas de la que depende para poder comenzar
}

TareasDB[10] = {
    'dur': (8,16),        # intervalo de duración en horas
    'dep': [7,8,9]       # tareas de la que depende para poder comenzar
}


In [30]:
# Generar muestras de las duraciones de las tareas

def sampleTSim(wTareasDB, n):
    TSim = [0] * 11
    random.seed()
    for i in range(1, n):
        TSim[i] = random.randint(wTareasDB[i]['dur'][0], wTareasDB[i]['dur'][1])
    return TSim
## 
    
sampleTSim(TareasDB,11)

[0, 50, 26, 27, 48, 17, 16, 25, 41, 51, 9]

In [9]:
# Calculo del tiempo de construcción

# Alg1 , "ir para atras"
# Tomo la tarea final
# Le sumo la duración max(tareas pre condicion)
# repito para la tarea que tenía ese max

def backtrackTareas(wTSim, wTDB, n):
    # print("corriendo con n: ",n)
    
    if n == 1:
        return [1]
    else:
        t_max_precond = 0
        max_precond = 0
        for j in wTDB[n]['dep']:
            if wTSim[j] > max_precond: 
                max_precond = wTSim[j]
                t_max_precond = j
            # end if
        # end for
        # print(j)
        prev = backtrackTareas(wTSim, wTDB, t_max_precond)
        prev.append(n)
        return prev
# end def

def calculoTiempoDeConstruccion(lista_tareas, wTSim):
    """
    listaTareas es un array con las tareas en el orden en que se ejecutaron
    """
    dur = 0
    for x in lista_tareas:
        dur = dur + wTSim[x]
    # end for
    return dur
# end def

def sortearX():
    ts = sampleTSim(TareasDB,11)
    tareas = backtrackTareas(ts, TareasDB, 10)
    return calculoTiempoDeConstruccion(tareas, ts)

In [33]:
muestras_promedio = []
muestras_devstd = []

def EstimacionMonteCarlo(n, FsortearX):
    t0 = perf_counter()
    random.seed()
    Xest = 0
    Vest = 0
    for x in range(n):
        Xi = FsortearX()
        Xest = Xest + Xi
        Vest = Vest + Xi**2
    # end for
    
    Xest = Xest / n
    Vest = Vest/(n*(n-1)) - Xest**2/(n-1)
    
    return (Xest, Vest, perf_counter()-t0)
# end def

In [34]:
EstimacionMonteCarlo(10**4, sortearX)

(164.5192, 0.01334965810180977, 0.2853529260000869)

## Resultados

### Corremos la simulación para diferentes tamaños de muestra

In [36]:
resultados = [
    'prom': [],
    'var': [],
    'devstd': [],
    't': []
]

tam = [10, 10**2, 10**3, 10**4, 10**5, 10**6]

for n in tam:
    (promedio, v, t) = EstimacionMonteCarlo(n, sortearX)
    devstd = math.sqrt( v*n)
    resultados['prom'].append(promedio)
    resultados['var'].append(v)
    resultados['devstd'].append(devstd)
    resultados['t'].append(t)
# end for

for n in range(tam):
    print(f"{resultados[n]['prom']:.2f} hs")

SyntaxError: invalid syntax (1707712899.py, line 2)